In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

from torchvision import transforms, datasets
from torchvision.models import resnet50
from torch.utils.data import DataLoader
from torchmetrics import Accuracy

import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt

from functions.dataset import *
from functions.model import *

C:\Users\Shadow\anaconda3\envs\torch_gpu\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
# Resnet version
model = resnet50(weights=None)
model.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
num_ftrs = model.fc.in_features
model.fc= nn.Linear(in_features=num_ftrs, out_features=2, bias=False)
model.to(device)

cuda:0


ResNet(
  (conv1): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [3]:
batch_size = 256 

data_train_dev = CustomDTS("datasets/ECG200_TRAIN.txt")
mean = data_train_dev.__getitem__(5)[0].mean()
std = data_train_dev.__getitem__(5)[0].std()

# transform = transforms.Compose([
#     transforms.ToTensor(), 
#     transforms.Normalize(mean, std) 
# ])

data_train_dev = CustomDTS("datasets/ECG200_TRAIN.txt")#, transform=transform)
data_test = CustomDTS("datasets/ECG200_TEST.txt")#, transform=transform)
data_train, data_dev = torch.utils.data.random_split(data_train_dev, [0.7, 0.3], generator=torch.Generator().manual_seed(42))
train_dl = DataLoader(data_train, batch_size=batch_size, shuffle=True)
dev_dl = DataLoader(data_dev, batch_size=batch_size, shuffle=False)
test_dl = DataLoader(data_test, batch_size=batch_size, shuffle=True)
train_eval_dl = {'train': train_dl, 'dev': dev_dl}

epochs = 10
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
metric = Accuracy(task="binary").to(device)

In [6]:
print(device)
model, best_wts, last_epoch_wts, losses, accuracies = train_dev_model(model, train_eval_dl, criterion, optimizer, device, metric, scheduler=None, epochs=epochs)

cuda:0


Epoch: 0, R_Loss: 0.0, R_Acc: 0:   0%|                                                           | 0/1 [00:00<?, ?it/s]


TypeError: to() received an invalid combination of arguments - got (BinaryAccuracy), but expected one of:
 * (torch.device device, torch.dtype dtype, bool non_blocking, bool copy, *, torch.memory_format memory_format)
 * (torch.dtype dtype, bool non_blocking, bool copy, *, torch.memory_format memory_format)
 * (Tensor tensor, bool non_blocking, bool copy, *, torch.memory_format memory_format)
